# **Evoltionary Timetable using Genetic Algorithm:** 

I have done it for one day, you can do the same for other days to get the complete TimeTable. Its simple.......

**Importing necessary libraries**

In [2]:
import pandas as pd
import numpy as np
import random
import copy

**Cleaning, loading data and preprocessing:** Before that I transformed the data into the format you can see below using the TimeTable and Course Allocation data. You can do that too, its pretty simple.

In [3]:
d1 = pd.read_excel("data/Input/Courses Allocation.xlsx")

d2 = pd.read_excel("data/Input/Courses Allocation.xlsx", sheet_name = "S&H")

d3 = pd.read_excel("data/Input/Courses Allocation.xlsx", sheet_name = "MG")

d1.drop(columns=["Unnamed: 1", "Unnamed: 3", "Unnamed: 6", "Unnamed: 7","Unnamed: 8", "Unnamed: 9"], inplace= True)

d1.drop([0,1,2],axis = 0, inplace = True)
d1.head(1)

,"FAST School of Computing, NUCES, Islamabad",Unnamed: 2,Unnamed: 4,Unnamed: 5
3,BS Batch 2022,Object Oriented Programming (CS),CS-2B,Mr. Shehreyar Rashid


In [ ]:
d1.columns = ["Batch_No", "Course_Name", "Section", "Instructor"]

d1.dropna(thresh = 2, axis = 0, inplace = True)

d1.reset_index(inplace = True)
del d1["index"]

d1["Cells"] = list(d1.itertuples(index=False, name=None))
d1.drop(d1.index[232:],axis = 0, inplace=True)
d1.head(1)

,Batch_No,Course_Name,Section,Instructor,Cells
0,BS Batch 2022,Object Oriented Programming (CS),CS-2B,Mr. Shehreyar Rashid,"(BS Batch 2022, Object Oriented Programming (C..."


In [ ]:
empty_mat= np.zeros((35,6))
List = d1["Cells"].to_list()
Rooms = ["C-301","C-302","C-303","C-304","C-305", "C-307","C-308","C-309","C-310","C-311","C-401","C-402","C-403","C-404","C-405","C-406","C-407","C-408","C-409","C-410","B-227","B-229","B-230","A-301","A-302","A-303","A-305","A-310","A-311","A-314","A-315","A-316","A-108","B-130","A-211"]
time = ["08:30-09:50", "10:00-11:20", "11:30-12:50", "01:00-02:20", "02:30-03:50", "03:55-05:15"]

d2.drop(columns=["Unnamed: 1", "Unnamed: 3", "Unnamed: 6"], inplace=True)
d2.drop([0,1,2],axis=0,inplace=True)
d2.dropna(thresh=2,axis=0,inplace=True)
d2.reset_index(inplace = True)
del d2["index"]



In [ ]:
d2.columns = ["Batch", "Course", "Section", "Instructor"]
d2["Cells"] = list(d2.itertuples(index=False, name=None))
d2.head(1)

,Batch,Course,Section,Instructor,Cells
0,BS Batch 2022,Differential Equations (CS),CS-2A,Hamda Khan,"(BS Batch 2022, Differential Equations (CS), C..."


In [ ]:
Lst = d2[d2['Course'].str.contains('Communication and Presentation Skills')].index
d2.drop(Lst, axis = 0, inplace = True)

d2.reset_index(inplace=True)
del d2["index"]
d2.head(1)

,Batch,Course,Section,Instructor,Cells
0,BS Batch 2022,Differential Equations (CS),CS-2A,Hamda Khan,"(BS Batch 2022, Differential Equations (CS), C..."


In [ ]:
d3.drop(columns=["Unnamed: 1", "Unnamed: 3", "Unnamed: 6"], inplace=True)
d3.drop([0,1,2],axis=0,inplace=True)
d3.dropna(thresh=2,axis=0,inplace=True)
d3.reset_index(inplace = True)
del d3["index"]
d3.columns = ["Batch", "Course", "Section", "Instructor"]
d3["Cells"] = list(d3.itertuples(index=False, name=None))
d3.head(1)

,Batch,Course,Section,Instructor,Cells
0,BS Batch 2021,Marketing Management (CS),CS-4A,Sadaf Zahra Usman,"(BS Batch 2021, Marketing Management (CS), CS-..."


**Concatinated data from all departments.**

In [ ]:
mainVal = d1["Cells"].to_list()
mainVal.extend(d2["Cells"].to_list() )
mainVal.extend(d3["Cells"].to_list())

In [ ]:
pd.DataFrame(mainVal).head(2)

,0,1,2,3
0,BS Batch 2022,Object Oriented Programming (CS),CS-2B,Mr. Shehreyar Rashid
1,BS Batch 2022,Object Oriented Programming (CS),CS-2C,Mr. Shehreyar Rashid


**Population generation**

In [ ]:
population = []
n=6*35
for i in range(50):

    chromosome = [[0 for j in range(6)] for i in range(35)]

    random_tuples = random.sample(List, n)
    cl=0
    for r in range(35):  # loop over the rows
        for c in range(6):  # loop over the columns


            chromosome[r][c]=random_tuples[cl]

            cl+=1

    population.append(chromosome)

In [ ]:
len(population)

50

**Main functions of genetic algorithm**

In [ ]:
#Max fitness will be 1140
#as 1st hard constraint can only go upto len(teachers)*6-->> 96*6
#2nd contraint can go upto len(sections)*6-->> 78*6
#3rd constraint can go upto len(teachers)-->96
#So total sum of these is 1140
#these numbers are according to my current timetable and course allocation data, your numbers can vary
def fitness(chromosome, teachers, sections):

    # Constraint for if teachers or students have more than 1 class at a time

    fitness=0
    looping_list = [teachers,sections]
    for i in looping_list:
        for c in range(6):
            for t in range(len(i)):
                t_present_count=0
                for r in range(35):
                    if chromosome[r][c][-1]==i[t]:
                        t_present_count+=1
                    if t_present_count>1:
                      break

                if t_present_count<=1:
                    fitness+=1


# if teacher have more than 2 classes in a day
    soft_cons1=0

    teachers_dict = {teacher: 0 for teacher in teachers}
    for t in range(len(teachers)):
      for c in range(6):
            for r in range(35):
                if chromosome[r][c][-1]==teachers[t]:
                  teachers_dict[teachers[t]]+=1

                if teachers_dict[teachers[t]]>2:
                  break
            if teachers_dict[teachers[t]]>2:
                  break

      if teachers_dict[teachers[t]]<=2:
            soft_cons1+=1

    return fitness+soft_cons1




In [ ]:
def selection(generation, teachers, sections):
    fit_val=[]
    for i in range(50):
        fit_val.append(fitness(generation[i], teachers, sections))
    top_10= np.argsort(np.array(fit_val))[::-1][:10]
    top_10_values= np.sort(fit_val)[:10]
    new_selection=[]
    for i in top_10:
        new_selection.append(generation[i])
    return new_selection, top_10_values,top_10[0]





In [ ]:
def mutation(generation):
    for r in range(10,50):
        for m in range(10):
            while(1):
                count=0
                rpoint= random.choice(List)
                for i in range(35):
                    if rpoint in generation[r][i]:
                        break
                    count+=1

                if count==35:
                    break
            n1= random.randint(0, 5)
            n2= random.randint(0, 34)
            generation[r][n2][n1]=rpoint
    return generation


In [ ]:
import random

def multi_point_crossover(mat1, mat2, num_points=15):
    """
    Perform multi-point crossover on two 2D matrices of the same shape.

    Args:
        mat1 (list of lists): The first matrix to be crossed over.
        mat2 (list of lists): The second matrix to be crossed over.
        num_points (int): The number of crossover points to use.

    Returns:
        A tuple containing the two new matrices created by crossover.
    """
    # Get the dimensions of the matrices
    rows, cols = len(mat1), len(mat1[0])

    # Choose random crossover points along each row
    available_points = list(range(cols))
    crossover_points = random.sample(available_points, num_points)
    crossover_points.sort()

    # Perform the crossover
    current_parent = 1
    child1 = [[mat1[r][c] if c < crossover_points[0] else mat2[r][c] for c in range(cols)] for r in range(rows)]
    child2 = [[mat2[r][c] if c < crossover_points[0] else mat1[r][c] for c in range(cols)] for r in range(rows)]
    for i in range(1, num_points):
        start = crossover_points[i - 1]
        end = crossover_points[i]
        current_parent = 1 - current_parent
        if current_parent == 0:
            child1 = [child1[r][:start] + mat2[r][start:end] + child1[r][end:] for r in range(rows)]
            child2 = [child2[r][:start] + mat1[r][start:end] + child2[r][end:] for r in range(rows)]
        else:
            child1 = [child1[r][:start] + mat1[r][start:end] + child1[r][end:] for r in range(rows)]
            child2 = [child2[r][:start] + mat2[r][start:end] + child2[r][end:] for r in range(rows)]

    return child1, child2


**Combining the GA functions to run our GA loop**

In [ ]:
#cross overing
#import pdb

teachers=[]
for r in range(len(List)):

    teachers.append(List[r][-1])

teachers=list(set(teachers))
sections=[]
for r in range(len(List)):

    sections.append(List[r][2])

sections=list(set(sections))
print(len(teachers),len(sections))
#selected,_ = selection(copy.deepcopy(population), teachers, sections)
iterations= 1000
x=0
while(x<=iterations):


    selected, max_fitness, max_idx = selection(copy.deepcopy(population), teachers, sections)
    if x%100==0:
      print(f'Iteration {x}',max_fitness)
      #Max fitness will be 1140
      #as 1st hard constraint can only go upto len(teachers)*6-->> 96*6
      #2nd contraint can go upto len(sections)*6-->> 78*6
      #3rd constraint can go upto len(teachers)-->96
      #So total sum of these is 1140
      #these numbers are according to my current timetable and course allocation data, your numbers can vary
    if (np.max(max_fitness)>=len(teachers)*6+len(sections)*6+len(teachers)):
      print('Constraints fulfilled')
      break

    for i in range(20):
        n1= random.randint(0, 9)
        n2= random.randint(0, 9)
        while(1):
            if(n2!=n1):
                break
            else:
                n2= random.randint(0, 9)
        p1,p2=multi_point_crossover(selected[n1], selected[n2],6)
        selected.append(p1)
        selected.append(p2)
    selected= mutation(selected)
    population=selected
    x+=1
    #pdb.set_trace()
    #check=fitness(selected, teachers, sections)






96 78
Iteration 0 [1069 1072 1072 1073 1073 1074 1074 1075 1075 1075]
Iteration 100 [1097 1099 1100 1100 1100 1101 1101 1101 1101 1102]
Iteration 200 [1102 1102 1102 1102 1103 1103 1103 1103 1104 1104]
Iteration 300 [1100 1101 1101 1101 1102 1102 1102 1102 1102 1103]
Iteration 400 [1099 1100 1102 1102 1102 1103 1103 1103 1103 1104]
Iteration 500 [1100 1100 1100 1101 1102 1102 1102 1103 1103 1103]
Iteration 600 [1098 1104 1104 1104 1105 1105 1105 1105 1105 1105]
Iteration 700 [1103 1104 1104 1104 1105 1105 1105 1105 1105 1105]
Iteration 800 [1101 1102 1104 1105 1105 1105 1105 1105 1106 1106]
Iteration 900 [1100 1101 1103 1104 1105 1106 1106 1106 1106 1106]
Iteration 1000 [1100 1103 1104 1104 1104 1106 1106 1107 1107 1107]


# Final time table

In [ ]:
final=pd.DataFrame(population[max_idx])
final.index = Rooms
final.columns = time
final.head(5)

,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
C-301,"(BS Batch 2021, Fundamentals of Software Engin...","(BS Batch 2022, Object Oriented Programming (C...","(BS Batch 2020, Statistical Modelling, CS-6A, ...","(BS Batch 2020, Process Mining and Simulation ...","(BS Batch 2019, Final Year Project-I (DS), DS-...",(BS (CS/DS/CY) Batch 2019 and Earlier Batches ...
C-302,"(BS Batch 2020, Applied Artificial Intelligenc...","(BS Batch 2020, Web Engineering (SE), SE-6P, M...",(BS Batch 2021 and Earlier Batches Repeat Cour...,"(BS Batch 2020, Web Engineering (SE), SE-6R, M...","(BS Batch 2022, Discrete Structures, AI-2D, Ms...","(BS Batch 2022, Object Oriented Programming (C..."
C-303,"(BS Batch 2020, Computer Networks (AI), AI-6K,...","(BS Batch 2019, Professional Practices in IT (...","(BS Batch 2020, Web Engineering (SE), SE-6P, M...","(BS Batch 2022, Introduction to Software Engin...","(BS Batch 2022, Object Oriented Programming (C...","(BS Batch 2020, Data Mining (DS), DS-6N, Dr. A..."
C-304,(BS Batch 2022 and Earlier Batches Repeat Cour...,(BS (CS/DS/CY) Batch 2019 and Earlier Batches ...,"(BS Batch 2020, Parallel and Distributed Compu...","(BS Batch 2021, Operating Systems (CS), CS-4A/...","(BS Batch 2020, Information Security (SE), SE-...",(BS Batch 2022 and Earlier Batches Repeat Cour...
C-305,"(BS Batch 2020, Information Security (SE), SE-...","(BS Batch 2022, Digital Logic Design (CS), CS-...",(BS Batch 2021 and Earlier Batches Repeat Cour...,"(BS Batch 2020, Parallel and Distributed Compu...","(BS Batch 2021, Fundamentals of Big Data Analy...","(BS Batch 2020, Artificial Intelligence (CS), ..."


In [1]:
#Saving our final result as Excel output
file_destination = 'data/Result/TimeTable.xlsx'
final.to_excel(file_destination, index=True)

print(f"DataFrame saved at {file_destination}")

DataFrame saved at data/Result/TimeTable.xlsx
